---
title: Dynamic Citation Tracker
subtitle: with ORCID and DoI.org API Scraping
---




In [2]:
import pandas as pd
import requests
from IPython.display import Markdown, JSON
from pathlib import Path
from rich import progress

# My ORCID
orcid_id = "0000-0001-5904-2887"
ORCID_RECORD_API = "https://pub.orcid.org/v3.0/"

# Download all of my ORCID records
print("Retrieving ORCID entries from API...")
response = requests.get(
    url=requests.utils.requote_uri(ORCID_RECORD_API + orcid_id),
    headers={"Accept": "application/json"},
)
response.raise_for_status()
orcid_record = response.json()


# Just to visualize in a notebook if need be
JSON(orcid_record)

Retrieving ORCID entries from API...


<IPython.core.display.JSON object>

In [3]:
# Resolve my DOIs from ORCID as references
# Shamelessly copied from:
# https://gist.github.com/brews/8d3b3ede15d120a86a6bd6fc43859c5e
import requests
import json


def fetchmeta(doi, fmt="reference", **kwargs):
    """Fetch metadata for a given DOI.

    Parameters
    ----------
    doi : str
    fmt : str, optional
        Desired metadata format. Can be 'dict' or 'bibtex'.
        Default is 'dict'.
    **kwargs :
        Additional keyword arguments are passed to `json.loads()` if `fmt`
        is 'dict' and you're a big JSON nerd.

    Returns
    -------
    out : str or dict or None
        `None` is returned if the server gives unhappy response. Usually
        this means the DOI is bad.

    Examples
    --------
    >>> fetchmeta('10.1016/j.dendro.2018.02.005')
    >>> fetchmeta('10.1016/j.dendro.2018.02.005', 'bibtex')

    References
    ----------
    https://www.doi.org/hb.html
    """
    # Parse args and setup the server response we want.
    accept_type = "application/"
    if fmt == "dict":
        accept_type += "citeproc+json"
    elif fmt == "bibtex":
        accept_type += "x-bibtex"
    elif fmt == "reference":
        accept_type = "text/x-bibliography; style=apa"
    else:
        raise ValueError(f"Unrecognized `fmt`: {fmt}")

    # Request data from server.
    url = "https://dx.doi.org/" + str(doi)
    header = {"accept": accept_type}
    r = requests.get(url, headers=header)

    # Format metadata if server response is good.
    out = None
    if r.status_code == 200:
        if fmt == "dict":
            out = json.loads(r.text, **kwargs)
        else:
            out = r.text
    return out


# -

# Extract metadata for each entry
df = []
for iwork in progress.track(
    orcid_record["activities-summary"]["works"]["group"], "Fetching reference data..."
):
    isummary = iwork["work-summary"][0]

    # Extract the DOI
    for ii in isummary["external-ids"]["external-id"]:
        if ii["external-id-type"] == "doi":
            year = isummary["publication-date"]["year"]["value"]
            doi = ii["external-id-value"]
            break
    df.append({"year": year, "doi": doi})
df = pd.DataFrame(df)

# Convert into a markdown string
# md = ["|Year|Publications|", "|===|===|"]
# for year, items in df.groupby("year", sort=False):
#     this_pubs = []
#     for _, item in items.iterrows():
#         this_pubs.append(f'{{cite}}`{item["doi"]}`')
#     md.append(f"|{year}|{', '.join(this_pubs)}|")
# mds = "\n".join(md)

# # +
# Uncomment to preview in a notebook
# Markdown(mds)
# -
# print(mds)
# df.groupby('year').sum()
# This will only work if this is run as a script
# path_out = Path(__file__).parent.parent / "_static/publications.txt"
# path_out.write_text(mds)
# print(f"Finished updating ORCID entries at: {path_out}")

/home/rtbs/.pyenv/versions/affinis/lib/python3.11/site-packages/rich/live.py:256: UserWarning: install "ipywidgets"
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

In [7]:
#| label: tbl:pubs
summary = df.assign(doi=lambda df: '@'+df.doi).groupby('year').agg(lambda x: " ".join(x)).sort_values(by='year',ascending=False)
print(summary.to_markdown())

|   year | doi                                                                                                                                 |
|-------:|:------------------------------------------------------------------------------------------------------------------------------------|
|   2025 | @10.13016/O252-LFMI                                                                                                                 |
|   2023 | @10.48550/arXiv.2311.04064 @10.48550/arXiv.2303.17820                                                                               |
|   2022 | @10.1007/s10845-021-01772-5                                                                                                         |
|   2021 | @10.1109/PacificVis52677.2021.00033 @10.36001/ijphm.2021.v12i1.2930 @10.1002/ail2.33 @10.1002/ail2.33 @10.1016/j.mfglet.2020.11.001 |
|   2020 | @10.1115/MSEC2020-8440 @10.1520/SSMS20200051 @10.1115/1.4045686 @10.3850/978-981-14-8593-0_5241-cd                     

In [5]:
print(summary.to_markdown())

|   year | doi                                                                                                                                 |
|-------:|:------------------------------------------------------------------------------------------------------------------------------------|
|   2025 | @10.13016/O252-LFMI                                                                                                                 |
|   2023 | @10.48550/arXiv.2311.04064 @10.48550/arXiv.2303.17820                                                                               |
|   2022 | @10.1007/s10845-021-01772-5                                                                                                         |
|   2021 | @10.1109/PacificVis52677.2021.00033 @10.36001/ijphm.2021.v12i1.2930 @10.1002/ail2.33 @10.1002/ail2.33 @10.1016/j.mfglet.2020.11.001 |
|   2020 | @10.1115/MSEC2020-8440 @10.1520/SSMS20200051 @10.1115/1.4045686 @10.3850/978-981-14-8593-0_5241-cd                     

|   year | doi                                                                                                                                 |
|-------:|:------------------------------------------------------------------------------------------------------------------------------------|
|   2025 | @10.13016/O252-LFMI                                                                                                                 |
|   2023 | @10.48550/arXiv.2311.04064 @10.48550/arXiv.2303.17820                                                                               |
|   2022 | @10.1007/s10845-021-01772-5                                                                                                         |
|   2021 | @10.1109/PacificVis52677.2021.00033 @10.36001/ijphm.2021.v12i1.2930 @10.1002/ail2.33 @10.1002/ail2.33 @10.1016/j.mfglet.2020.11.001 |
|   2020 | @10.1115/MSEC2020-8440 @10.1520/SSMS20200051 @10.1115/1.4045686 @10.3850/978-981-14-8593-0_5241-cd                                  |
|   2019 | @10.6028/jres.124.029 @10.36001/phmconf.2019.v11i1.792 @10.1115/DETC2019-98429 @10.1115/1.4044105 @10.1115/1.4044105                |
|   2018 | @10.1115/MSEC2018-6492 @10.1520/SSMS20180018                                                                                        |
|   2017 | @10.1109/BigData.2017.8258120 @10.1115/1.4037344 @10.1007/978-3-319-66923-6_50                                                      |
|   2016 | @10.1115/DETC2016-59775 |

![](#tbl:pubs)